In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from rdflib import Graph, Namespace
from rdflib.namespace import RDF

# === 1. Globale Variablen ===
instance_file_path = None
result_graph = None
selected_system_uri = None
selected_system_id = None

# === 2. UI: Dateiauswahl ===
option_selector = widgets.ToggleButtons(
    options=[('Use example instance', 'example'), ('Upload your own', 'upload')],
    description='Choose input:',
    style={'description_width': 'initial'}
)

upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

output = widgets.Output()
shacl_output = widgets.Output()
query_interface_output = widgets.Output()
selection_output = widgets.Output()

# === 3. SHACL Namespaces & Shapes ===
SH = Namespace("http://www.w3.org/ns/shacl#")
SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
shapes_dir = "Shapes"
shape_files = [
    "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
    "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
    "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
    "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
    "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
    "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
    "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
    "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
    "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
    "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
    "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl", "Y4-shapes.ttl"
]

# === 4. SHACL-Regeln anwenden ===
def apply_shacl_rules(file_path):
    g_instance = Graph()
    try:
        g_instance.parse(file_path, format="turtle")
    except Exception as e:
        with shacl_output:
            print(f"❌ Error parsing instance file: {e}")
        return None

    triples_before = len(g_instance)
    with shacl_output:
        print(f"📥 Parsed instance file: {file_path}")
        print(f"📊 Triples before SHACL: {triples_before}")
        print("🔍 Applying SHACL rules...")

    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_file}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error parsing shape file {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                    with shacl_output:
                        print(f"✅ Applied rule from {shape_file}")
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Rule error ({shape_file}): {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print(f"✅ All rules applied. New triples: {triples_after - triples_before}")

    return g_instance

# === 5. SPARQL UI ===
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>
SELECT DISTINCT ?system WHERE { ?system a swemls:System . }
""",
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

run_query_button = widgets.Button(
    description='Run Query',
    button_style='primary'
)

system_selector = widgets.Dropdown(description="Select system:")
confirm_button = widgets.Button(description="Confirm selection", button_style='success')

def on_query_run(b):
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if not result_graph:
        with query_interface_output:
            print("⚠️ No RDF graph loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ SPARQL error: {e}")
        return

    systems = [(str(row.system).split("/")[-1], str(row.system)) for row in results]
    if not systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = systems
    with query_interface_output:
        print(f"✅ Found {len(systems)} system(s).")
    display(system_selector, confirm_button)

def on_confirm_selection(b):
    global selected_system_id, selected_system_uri
    selected_system_id = system_selector.label
    selected_system_uri = system_selector.value
    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_system_id}")

run_query_button.on_click(on_query_run)
confirm_button.on_click(on_confirm_selection)

# === 6. Fortsetzen-Button (führt alles aus) ===
continue_button = widgets.Button(description="Continue", button_style="info")

def run_main_pipeline():
    global result_graph
    output.clear_output()
    shacl_output.clear_output()

    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    if not instance_file_path.startswith("Instance_Files/"):
        with output:
            print("🔄 Applying SHACL rules to uploaded file...")
        result_graph = apply_shacl_rules(instance_file_path)
        display(shacl_output)
    else:
        with output:
            print(f"📥 Loading example file: {instance_file_path}")
        result_graph = Graph()
        try:
            result_graph.parse(instance_file_path, format="turtle")
        except Exception as e:
            with output:
                print(f"❌ Error loading file: {e}")
            return

    display(query_input, run_query_button, query_interface_output, selection_output)

def on_continue_clicked(b):
    run_main_pipeline()

continue_button.on_click(on_continue_clicked)

# === 7. Upload- und Auswahlhandling ===
def on_selection_change(change):
    output.clear_output()
    if change['new'] == 'example':
        global instance_file_path
        instance_file_path = os.path.join("Instance_Files", "swemls-instances.ttl")
        with output:
            print(f"✅ Example file selected: {instance_file_path}")
    elif change['new'] == 'upload':
        with output:
            print("📁 Please upload a TTL file:")
        display(upload_widget)

def on_file_upload(change):
    global instance_file_path
    uploaded_file = list(upload_widget.value.values())[0]
    instance_file_path = uploaded_file['metadata']['name']
    with open(instance_file_path, 'wb') as f:
        f.write(uploaded_file['content'])
    output.clear_output()
    with output:
        print(f"✅ Uploaded and saved: {instance_file_path}")

option_selector.observe(on_selection_change, names='value')
upload_widget.observe(on_file_upload, names='value')

# === 8. Anzeige der UI ===
display(option_selector, output, continue_button)


NameError: name 'lol' is not defined